In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import cloudscraper
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

path = "chromedriver-win32/chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument("--headless")
service = Service(path)
# browser = webdriver.Chrome(options = chrome_options, service = service)
browser = webdriver.Chrome()
   
browser.get("https://www.imdb.com/")
search_box = browser.find_element("id", "suggestion-search")

search_box.clear()
search_box.send_keys("Minions")

search_btn = browser.find_element("id", "suggestion-search-button")
search_btn.click()

page = browser.page_source
soup = BeautifulSoup(page, "html.parser")

link_tag = soup.find('a', class_='ipc-metadata-list-summary-item__t')
if link_tag:
    link = link_tag['href']
    full_link = f"https://www.imdb.com{link}"
    print("Menemukan link: ", full_link)
    
    # Navigate to the movie page
    browser.get(full_link)
    
    # Get the page source and parse it with BeautifulSoup
    page = browser.page_source
    soup = BeautifulSoup(page, "html.parser")
    
    # Scrape the desired element (e.g., Top picks)
    top_picks_tag = soup.find('a', href=re.compile('/what-to-watch/top-picks'))
    if top_picks_tag:
        top_picks_link = top_picks_tag['href']
        full_top_picks_link = f"https://www.imdb.com{top_picks_link}"
        print("Menemukan Top Picks link: ", full_top_picks_link)
        
        # Navigate to the Top Picks page
        browser.get(full_top_picks_link)
        
        # Wait for the page to load
        browser.implicitly_wait(5)
        
        # Get the page source and parse it with BeautifulSoup
        page = browser.page_source
        soup = BeautifulSoup(page, "html.parser")
        
        # Scrape the movie titles and links
        movie_tags = soup.find_all('a', class_='ipc-poster-card__title ipc-poster-card__title--clamp-2 ipc-poster-card__title--clickable')
        movies = []
        for tag in movie_tags:
            title = tag.find('span', {'data-testid': 'title'}).text
            movie_link = f"https://www.imdb.com{tag['href']}"
            movies.append({"Title": title, "Link": movie_link})
            print(f"Title: {title}")
            print(f"Link: {movie_link}")
            print(f"")
        
        # Convert the list of dictionaries to a DataFrame and save as CSV
        df = pd.DataFrame(movies)
        df.to_csv('title.csv', index=False)
        print("CSV file title.csv sudah dibuat.")
    else:
        print("Top Picks link tidak ditemukan")
else:
    print("Link tidak ditemukan")

Menemukan link:  https://www.imdb.com/title/tt2293640/?ref_=fn_al_tt_1
Menemukan Top Picks link:  https://www.imdb.com/what-to-watch/top-picks/?ref_=tt_tpks_sm
Title: Despicable Me 2
Link: https://www.imdb.com/title/tt1690953/?ref_=watch_tpks_tt_t_1_pd_tp1_pbr_ic

Title: Despicable Me
Link: https://www.imdb.com/title/tt1323594/?ref_=watch_tpks_tt_t_2_pd_tp1_pbr_ic

Title: Hotel Transylvania
Link: https://www.imdb.com/title/tt0837562/?ref_=watch_tpks_tt_t_3_pd_tp1_pbr_ic

Title: The Angry Birds Movie
Link: https://www.imdb.com/title/tt1985949/?ref_=watch_tpks_tt_t_4_pd_tp1_pbr_ic

Title: Ralph Breaks the Internet
Link: https://www.imdb.com/title/tt5848272/?ref_=watch_tpks_tt_t_5_pd_tp1_pbr_ic

Title: Wreck-It Ralph
Link: https://www.imdb.com/title/tt1772341/?ref_=watch_tpks_tt_t_6_pd_tp1_pbr_ic

Title: Cars 3
Link: https://www.imdb.com/title/tt3606752/?ref_=watch_tpks_tt_t_7_pd_tp1_pbr_ic

Title: Ice Age
Link: https://www.imdb.com/title/tt0268380/?ref_=watch_tpks_tt_t_8_pd_tp1_pbr_ic

T

In [2]:
movies_data = []
for tag in movie_tags:
    title = tag.find('span', {'data-testid': 'title'}).text
    movie_link = f"https://www.imdb.com{tag['href']}"
    
    # Navigate to the movie's page
    browser.get(movie_link)
    browser.implicitly_wait(5)
    movie_page = browser.page_source
    movie_soup = BeautifulSoup(movie_page, "html.parser")
    
    # Scrape box office information
    box_office = {
        "Budget": "0",
        "Gross US & Canada": "0",
        "Opening weekend US & Canada": "0",
        "Gross worldwide": "0"
    }
    box_office_tags = movie_soup.find_all('li', {'data-testid': re.compile('title-boxoffice-*')})
    for box_office_tag in box_office_tags:
        label = box_office_tag.find('span', class_='ipc-metadata-list-item__label').text.strip()
        value = box_office_tag.find('span', class_='ipc-metadata-list-item__list-content-item').text.strip()
        # Menghilangkan simbol $ dan (estimated)
        value = re.sub(r'[$, \(estimated\)]', '', value)
        if label in box_office:
            box_office[label] = value
    
    # Scrape technical specs information
    technical_specs = {
        "Runtime": "0",
        "Color": "0",
        "Sound mix": "0",
        "Aspect ratio": "0"
    }
    tech_specs_tags = movie_soup.find_all('li', {'data-testid': re.compile('title-techspec-*')})
    for tech_specs_tag in tech_specs_tags:
        label = tech_specs_tag.find('span', class_='ipc-metadata-list-item__label').text.strip()
        content_container = tech_specs_tag.find('div', class_='ipc-metadata-list-item__content-container')
        if content_container:
            value = content_container.get_text(separator=" ").strip()
            if label == "Runtime":
                parts = value.split()
                runtime_minutes = 0
                for i in range(len(parts)):
                    if 'h' in parts[i]:
                        hours = int(parts[i-1].replace('h', '')) if i > 0 else 0
                        runtime_minutes += hours * 60
                    elif 'min' in parts[i]:
                        minutes_str = parts[i-1] if i > 0 else parts[i]  # Ambil bagian sebelum "min"
                        minutes = int(minutes_str.replace('min', '')) if minutes_str.isdigit() else 0
                        runtime_minutes += minutes
                value = str(runtime_minutes)
            if label in technical_specs:
                technical_specs[label] = value

    
    # Combine the data
    movie_data = {
        "Title": title,
        **box_office,
        **technical_specs
    }
    movies_data.append(movie_data)
    
    print(f"Title: {title}")
    print(f"Box Office: {box_office}")
    print(f"Technical Specs: {technical_specs}")
    print(f"------------------------------------------")

# Buat DataFrame dari data movies_data
df = pd.DataFrame(movies_data)

# Isi nilai yang hilang dengan 0
df.fillna("0", inplace=True)

# Simpan DataFrame ke dalam file CSV dengan nama kolom yang diinginkan
df.to_csv('detailed_film.csv', index=False)

print("Data telah disimpan ke dalam file CSV dengan nama detailed_film.csv")


Title: Despicable Me 2
Box Office: {'Budget': '76000000', 'Gross US & Canada': '368065385', 'Opening weekend US & Canada': '83517315', 'Gross worldwide': '970766005'}
Technical Specs: {'Runtime': '98', 'Color': 'Color', 'Sound mix': 'Dolby Digital Datasat SDDS Dolby Atmos Dolby Surround 7.1 Auro 11.1', 'Aspect ratio': '1.85 : 1'}
------------------------------------------
Title: Despicable Me
Box Office: {'Budget': '69000000', 'Gross US & Canada': '251683815', 'Opening weekend US & Canada': '56397125', 'Gross worldwide': '543284256'}
Technical Specs: {'Runtime': '95', 'Color': 'Color', 'Sound mix': 'DTS Dolby Digital SDDS', 'Aspect ratio': '1.85 : 1'}
------------------------------------------
Title: Hotel Transylvania
Box Office: {'Budget': '85000000', 'Gross US & Canada': '148313048', 'Opening weekend US & Canada': '42522194', 'Gross worldwide': '358375603'}
Technical Specs: {'Runtime': '91', 'Color': 'Color', 'Sound mix': 'SDDS Datasat Dolby Digital Dolby Atmos', 'Aspect ratio': '1.

In [4]:
# Baca file CSV pertama
df1 = pd.read_csv('title.csv')

# Baca file CSV kedua
df2 = pd.read_csv('detailed_film.csv')

# Gabungkan kedua DataFrame berdasarkan kolom "Title"
merged_df = pd.merge(df1, df2, on='Title', how='inner')

# Simpan DataFrame hasil penggabungan ke dalam file CSV
merged_df.to_csv('merge_data_film.csv', index=False)

print("Data telah digabungkan dan disimpan dalam file CSV dengan nama merge_data_film.csv")


Data telah digabungkan dan disimpan dalam file CSV dengan nama merge_data_film.csv
